In [485]:
import pandas as pd
import os
import numpy as np
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
import json


In [486]:
def parse_json_file(json_file):
    """
    Função para converter arquivo JSON de sentimentos em um DataFrame.
    
    Parâmetros:
    json_file -- caminho para o arquivo JSON ou dados JSON em formato de string
    
    Retorno:
    Um DataFrame pandas com duas colunas: sentiment e confidence
    """
    # Lista para armazenar os resultados
    resultados = []
    
    # Verifica se o input é um caminho de arquivo ou uma string JSON
    if isinstance(json_file, str) and (json_file.endswith('.json') or '/' in json_file):
        # Lê o arquivo linha por linha
        with open(json_file, 'r') as file:
            linhas = file.readlines()
    else:
        # Assume que é uma string com linhas JSON
        linhas = json_file.strip().split('\n')
    
    # Processa cada linha
    for linha in linhas:
        try:
            # Carrega o JSON externo
            obj_externo = json.loads(linha)
            
            # Obtém a string de resposta
            resposta_str = obj_externo.get('response', '{}')
            
            # Remove possíveis espaços em branco extras
            resposta_str = resposta_str.strip()
            
            # Carrega o JSON interno
            try:
                # Tenta carregar como objeto JSON
                obj_interno = json.loads(resposta_str)
            except json.JSONDecodeError:
                # Se falhar, pode ser porque o JSON está dentro de uma string
                if resposta_str.startswith('"') and resposta_str.endswith('"'):
                    resposta_str = resposta_str[1:-1]
                obj_interno = json.loads(resposta_str)
            
            # Extrai o sentimento e a confiança
            sentimento = obj_interno.get('sentiment', '')
            confianca = obj_interno.get('confidence', 0.0)
            
            # Normaliza o sentimento para minúsculas para consistência
            sentimento = sentimento.lower()
            
            # Adiciona à lista de resultados
            resultados.append({'sentiment': sentimento, 'confidence': confianca})
            
        except Exception as e:
            print(f"Erro ao processar linha: {linha}")
            print(f"Erro: {e}")
    
    # Cria o DataFrame
    df = pd.DataFrame(resultados)
    
    return df

In [487]:
jira_data = pd.read_csv(r'C:\Users\breno\OneDrive\Documentos\GitHub\stnl-dataminer-api\studycase\data\jira.csv')

jira_sentiment_data =  parse_json_file(r"C:\Users\breno\OneDrive\Documentos\GitHub\stnl-dataminer-api\studycase\sentiment\results\jira.json")

In [488]:
jira_data.head()

issue_id issue_key project priority     status         assignee  \
0     10074  CSTONE-1  CSTONE   Medium  Concluído     Johny Arriel   
1     10075  CSTONE-2  CSTONE   Medium  Concluído  Luiza Marcondes   
2     10076  CSTONE-3  CSTONE   Medium  Concluído     Johny Arriel   
3     10077  CSTONE-4  CSTONE   Medium  Concluído  Luiza Marcondes   
4     10078  CSTONE-5  CSTONE   Medium  Concluído  Luiza Marcondes   

           creator                     created                     updated  \
0  Luiza Marcondes  2024-05-29 14:10:58.526+00  2024-06-10 23:13:30.458+00   
1  Luiza Marcondes  2024-05-29 14:11:01.651+00  2024-05-29 14:11:04.596+00   
2  Luiza Marcondes  2024-05-29 14:11:04.461+00  2024-05-29 14:11:07.681+00   
3  Luiza Marcondes  2024-05-29 14:11:07.557+00  2024-06-18 18:56:10.673+00   
4  Luiza Marcondes  2024-05-29 14:11:10.063+00  2024-07-15 12:50:28.082+00   

   issuetype                              issuetype_description  \
0   História  Funcionalidade ou recurso expressado como um o...   
1  Subtarefa  Uma pequena parte do trabalho que faz parte de...   
2  Subtarefa  Uma pequena parte do trabalho que faz parte de...   
3  Subtarefa  Uma pequena parte do trabalho que faz parte de...   
4  Subtarefa  Uma pequena parte do trabalho que faz parte de...   

                           summary  \
0              Realizar setup Jira   
1                 Configurar board   
2   Registrar usuários do projeto    
3  Integrar com repositório github   
4           Outras automações Jira   

                                         description  \
0  Como  um membro da equipe responsável pela con...   
1  Configurar e personalizar um novo projeto no J...   
2   Registrar usuários do projeto Stone Lab no Jira.   
3             Fazer integração do Jira com o GitHub.   
4  Alguns itens para explorar: Criação de tarefas...   

                                          all_fields commits  \
0  {"Rank": "0|i000ff:", "Team": null, "votes": {...      []   
1  {"Rank": "0|i000dz:", "Team": null, "votes": {...      []   
2  {"Rank": "0|i000e7:", "Team": null, "votes": {...      []   
3  {"Rank": "0|i000ef:", "Team": null, "votes": {...      []   
4  {"Rank": "0|i000ry:d", "Team": null, "votes": ...      []   

                      time_mined  
0  2025-03-24 16:33:47.196763+00  
1  2025-03-24 16:33:46.385039+00  
2  2025-03-24 16:33:45.275883+00  
3  2025-03-24 16:33:44.023845+00  
4  2025-03-24 16:33:43.237916+00

In [489]:
jira_sentiment_data.head()

sentiment  confidence
0  positive        0.90
1   neutral        0.90
2   neutral        0.90
3   neutral        0.90
4  positive        0.85

In [490]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from typing import Dict, Optional

class JiraSentimentAnalyzer:
    """Analisador de sentimentos para dados de issues do Jira.
    
    Esta classe oferece funcionalidades para analisar e visualizar dados de sentimentos
    associados a issues do Jira, incluindo distribuições gerais, evolução temporal,
    análise de produtividade e sobrecarga de trabalho por colaborador.
    
    Attributes:
        COLOR_MAP (Dict[str, str]): Mapeamento de cores para diferentes sentimentos:
            - positive: verde (#2ecc71)
            - neutral: cinza (#95a5a6)
            - negative: vermelho (#e74c3c)
        jira_df (pd.DataFrame): DataFrame contendo os dados do Jira.
        sentiment_df (pd.DataFrame): DataFrame contendo os dados de sentimentos.
        merged_data (pd.DataFrame): DataFrame resultante da mesclagem dos dados do Jira e de sentimentos.
    """

    COLOR_MAP = {
        'positive': '#2ecc71',
        'neutral': '#95a5a6',
        'negative': '#e74c3c'
    }
    
    def __init__(self, jira_df: pd.DataFrame, sentiment_df: pd.DataFrame):
        """Inicializa o analisador de sentimentos do Jira.
        
        Args:
            jira_df (pd.DataFrame): DataFrame contendo os dados brutos do Jira.
            sentiment_df (pd.DataFrame): DataFrame contendo os dados de análise de sentimentos.
        """
        self.jira_df = jira_df
        sentiment_df['sentiment'] = sentiment_df['sentiment'].str.lower()
        self.sentiment_df = sentiment_df
        self.merged_data = self._merge_dataframes()
        
    def _merge_dataframes(self) -> pd.DataFrame:
        """Mescla os DataFrames de Jira e sentimentos.
        
        Combina os dados do Jira com os dados de sentimentos, adicionando colunas de
        sentimento e confiança, além de converter a coluna de data para o formato datetime.
        
        Returns:
            pd.DataFrame: DataFrame mesclado contendo dados do Jira e sentimentos.
        """
        merged = self.jira_df.copy()
        merged['sentiment'] = self.sentiment_df['sentiment']
        merged['confidence'] = self.sentiment_df['confidence']
        merged['created'] = pd.to_datetime(merged['created'])
        return merged

    def plot_overall_sentiment_distribution(self):
        """Gera gráfico de pizza mostrando a distribuição geral de sentimentos.
        
        Cria uma visualização da distribuição geral de sentimentos (positivo, neutro, negativo)
        em todo o conjunto de dados.
        
        Returns:
            plotly.graph_objs._figure.Figure: Gráfico de pizza com a distribuição de sentimentos.
        """
        fig = px.pie(
            self.merged_data,
            names='sentiment',
            title='Overall Project Sentiment Distribution',
            color='sentiment',
            color_discrete_map=self.COLOR_MAP
        )
        fig.update_layout(
            showlegend=True,
            legend_title='Sentiments'
        )
        return fig

    def plot_sentiment_by_issue_type(self):
        """Gera gráfico de barras agrupadas mostrando sentimentos por tipo de issue.
        
        Cria uma visualização da distribuição de sentimentos segmentada por diferentes
        tipos de issues do Jira.
        
        Returns:
            plotly.graph_objs._figure.Figure: Gráfico de barras com distribuição de sentimentos por tipo de issue.
        """
        sentiment_by_type = pd.crosstab(
            self.merged_data['issuetype'],
            self.merged_data['sentiment']
        )
        
        fig = px.bar(
            sentiment_by_type,
            title='Sentiment Distribution by Issue Type',
            barmode='group',
            color_discrete_map=self.COLOR_MAP
        )
        fig.update_layout(
            xaxis_title='Issue Type',
            yaxis_title='Number of Issues',
            legend_title='Sentiment'
        )
        return fig

    def plot_temporal_sentiment_evolution(self):
        """Gera gráfico de linha mostrando a evolução dos sentimentos ao longo do tempo.
        
        Cria uma visualização da evolução temporal dos diferentes sentimentos (positivo, 
        neutro, negativo) ao longo dos meses.
        
        Returns:
            plotly.graph_objs._figure.Figure: Gráfico de linha com evolução temporal dos sentimentos.
        """
        temporal_analysis = (
            self.merged_data
            .groupby([pd.Grouper(key='created', freq='M'), 'sentiment'])
            .size()
            .unstack(fill_value=0)
        )
        
        fig = go.Figure()
        
        for sentiment in temporal_analysis.columns:
            fig.add_trace(
                go.Scatter(
                    x=temporal_analysis.index,
                    y=temporal_analysis[sentiment],
                    name=sentiment,
                    mode='lines+markers',
                    line=dict(color=self.COLOR_MAP[sentiment], width=2),
                    marker=dict(size=6)
                )
            )
        
        fig.update_layout(
            title='Temporal Evolution of Sentiments',
            xaxis_title='Date',
            yaxis_title='Number of Issues',
            hovermode='x unified'
        )
        return fig

    def plot_collaborator_sentiment_evolution(self, collaborator: str):
        """Gera gráfico de linha mostrando a evolução dos sentimentos para um colaborador específico.
        
        Args:
            collaborator (str): Nome do colaborador para análise.
            
        Returns:
            plotly.graph_objs._figure.Figure: Gráfico de linha com evolução dos sentimentos do colaborador.
        """
        collaborator_data = self.merged_data[self.merged_data['assignee'] == collaborator]
        
        temporal_analysis = (
            collaborator_data
            .groupby([pd.Grouper(key='created', freq='M'), 'sentiment'])
            .size()
            .unstack(fill_value=0)
        )
        
        fig = go.Figure()
        
        for sentiment in temporal_analysis.columns:
            fig.add_trace(
                go.Scatter(
                    x=temporal_analysis.index,
                    y=temporal_analysis[sentiment],
                    name=sentiment,
                    mode='lines+markers',
                    line=dict(color=self.COLOR_MAP[sentiment], width=2),
                    marker=dict(size=6)
                )
            )
        
        fig.update_layout(
            title=f'Sentiment Evolution for {collaborator}',
            xaxis_title='Date',
            yaxis_title='Number of Issues',
            hovermode='x unified'
        )
        return fig

    def plot_productivity_evolution(self, collaborator: Optional[str] = None):
        """Gera gráfico de evolução da produtividade ao longo do tempo por pessoa.
        
        Visualiza a taxa de conclusão de issues ao longo do tempo, segmentada por colaborador.
        
        Args:
            collaborator (Optional[str]): Nome do colaborador para filtrar dados. 
                Se None, mostra dados para todos os colaboradores.
                
        Returns:
            plotly.graph_objs._figure.Figure: Gráfico de linha com evolução da produtividade.
        """
        data = self.merged_data if collaborator is None else \
            self.merged_data[self.merged_data['assignee'] == collaborator]
        
        monthly_metrics = (
            data.groupby([pd.Grouper(key='created', freq='ME'), 'assignee'])
            .agg({
                'issue_key': 'count',
                'status': lambda x: (x == 'Concluído').sum(),
            })
            .reset_index()
        )
        
        monthly_metrics['completion_rate'] = monthly_metrics['status'] / monthly_metrics['issue_key']
        
        fig = go.Figure()
        
        for assignee in monthly_metrics['assignee'].unique():
            person_data = monthly_metrics[monthly_metrics['assignee'] == assignee]
            
            fig.add_trace(
                go.Scatter(
                    x=person_data['created'],
                    y=person_data['completion_rate'],
                    name=assignee,
                    mode='lines+markers',
                    hovertemplate=(
                        "<b>%{x}</b><br>" +
                        "Taxa de Conclusão: %{y:.1%}<br>" +
                        "Pessoa: " + assignee
                    )
                )
            )
        
        fig.update_layout(
            height=500,
            width=1000,
            title_text='Evolução da Produtividade por Pessoa',
            xaxis_title='Data',
            yaxis_title='Taxa de Conclusão',
            yaxis_tickformat='.0%',
            hovermode='closest'
        )
        
        return fig

    def plot_workload_completion_relation(self, collaborator: Optional[str] = None):
        """Gera gráfico de relação entre carga de trabalho e taxa de conclusão.
        
        Cria uma visualização da relação entre o volume médio de issues atribuídas e a taxa de 
        conclusão para cada colaborador, com cores indicando a proporção de sentimentos positivos.
        
        Args:
            collaborator (Optional[str]): Nome do colaborador para filtrar dados.
                Se None, mostra dados para todos os colaboradores.
                
        Returns:
            plotly.graph_objs._figure.Figure: Gráfico de dispersão relacionando carga de trabalho e taxa de conclusão.
        """
        data = self.merged_data if collaborator is None else \
            self.merged_data[self.merged_data['assignee'] == collaborator]
        
        monthly_metrics = (
            data.groupby([pd.Grouper(key='created', freq='ME'), 'assignee'])
            .agg({
                'issue_key': 'count',
                'status': lambda x: (x == 'Concluído').sum(),
                'sentiment': lambda x: (x == 'positive').mean()
            })
            .reset_index()
        )
        
        monthly_metrics['completion_rate'] = monthly_metrics['status'] / monthly_metrics['issue_key']
        
        avg_metrics = monthly_metrics.groupby('assignee').agg({
            'issue_key': 'mean',
            'completion_rate': 'mean',
            'sentiment': 'mean'
        }).reset_index()
        
        fig = go.Figure()
        
        fig.add_trace(
            go.Scatter(
                x=avg_metrics['issue_key'],
                y=avg_metrics['completion_rate'],
                mode='markers+text',
                marker=dict(
                    size=15,
                    color=avg_metrics['sentiment'],
                    colorscale='RdYlGn',
                    showscale=True,
                    colorbar=dict(title="% Sentimentos Positivos")
                ),
                text=avg_metrics['assignee'],
                textposition="top center",
                hovertemplate=(
                    "<b>%{text}</b><br>" +
                    "Média de Issues: %{x:.1f}<br>" +
                    "Taxa de Conclusão: %{y:.1%}<br>" +
                    "Sentimentos Positivos: %{marker.color:.1%}"
                )
            )
        )
        
        fig.update_layout(
            height=500,
            width=1000,
            title_text='Relação entre Carga e Taxa de Conclusão',
            xaxis_title='Média de Issues por Mês',
            yaxis_title='Taxa de Conclusão',
            yaxis_tickformat='.0%',
            hovermode='closest'
        )
        
        return fig

    def plot_sentiment_evolution(self, collaborator: Optional[str] = None):
        """Gera gráfico de evolução dos sentimentos positivos ao longo do tempo.
        
        Visualiza a proporção de sentimentos positivos ao longo do tempo para cada colaborador.
        
        Args:
            collaborator (Optional[str]): Nome do colaborador para filtrar dados.
                Se None, mostra dados para todos os colaboradores.
                
        Returns:
            plotly.graph_objs._figure.Figure: Gráfico de linha com evolução dos sentimentos positivos.
        """
        data = self.merged_data if collaborator is None else \
            self.merged_data[self.merged_data['assignee'] == collaborator]
        
        monthly_metrics = (
            data.groupby([pd.Grouper(key='created', freq='ME'), 'assignee'])
            .agg({
                'sentiment': lambda x: (x == 'positive').mean()
            })
            .reset_index()
        )
        
        fig = go.Figure()
        
        for assignee in monthly_metrics['assignee'].unique():
            person_data = monthly_metrics[monthly_metrics['assignee'] == assignee]
            
            fig.add_trace(
                go.Scatter(
                    x=person_data['created'],
                    y=person_data['sentiment'],
                    name=assignee,
                    mode='lines+markers',
                    hovertemplate=(
                        "<b>%{x}</b><br>" +
                        "Sentimentos Positivos: %{y:.1%}<br>" +
                        "Pessoa: " + assignee
                    )
                )
            )
        
        fig.update_layout(
            height=500,
            width=1000,
            title_text='Evolução dos Sentimentos Positivos',
            xaxis_title='Data',
            yaxis_title='Taxa de Sentimentos Positivos',
            yaxis_tickformat='.0%',
            hovermode='closest'
        )
        
        return fig

    def plot_average_productivity(self, collaborator: Optional[str] = None):
        """Gera gráfico de média de produtividade por pessoa.
        
        Cria uma visualização da taxa média de conclusão de issues por colaborador,
        com cores indicando a proporção de sentimentos positivos.
        
        Args:
            collaborator (Optional[str]): Nome do colaborador para filtrar dados.
                Se None, mostra dados para todos os colaboradores.
                
        Returns:
            plotly.graph_objs._figure.Figure: Gráfico de barras com média de produtividade por pessoa.
        """
        data = self.merged_data if collaborator is None else \
            self.merged_data[self.merged_data['assignee'] == collaborator]
        
        monthly_metrics = (
            data.groupby([pd.Grouper(key='created', freq='ME'), 'assignee'])
            .agg({
                'issue_key': 'count',
                'status': lambda x: (x == 'Concluído').sum(),
                'sentiment': lambda x: (x == 'positive').mean()
            })
            .reset_index()
        )
        
        monthly_metrics['completion_rate'] = monthly_metrics['status'] / monthly_metrics['issue_key']
        
        avg_metrics = monthly_metrics.groupby('assignee').agg({
            'completion_rate': 'mean',
            'sentiment': 'mean'
        }).reset_index()
        
        fig = go.Figure()
        
        fig.add_trace(
            go.Bar(
                x=avg_metrics['assignee'],
                y=avg_metrics['completion_rate'],
                marker=dict(
                    color=avg_metrics['sentiment'],
                    colorscale='RdYlGn',
                    showscale=True,
                    colorbar=dict(title="% Sentimentos Positivos")
                ),
                hovertemplate=(
                    "<b>%{x}</b><br>" +
                    "Taxa Média de Conclusão: %{y:.1%}<br>" +
                    "Média de Sentimentos Positivos: %{marker.color:.1%}"
                )
            )
        )
        
        fig.update_layout(
            height=500,
            width=1000,
            title_text='Média de Produtividade por Pessoa',
            xaxis_title='Pessoa',
            xaxis_tickangle=45,
            yaxis_title='Taxa Média de Conclusão',
            yaxis_tickformat='.0%',
            hovermode='closest'
        )
        
        return fig

    def plot_team_productivity_timeline(self):
        """Gera gráfico de linha mostrando métricas de produtividade da equipe ao longo do tempo.
        
        Cria uma visualização com duas métricas ao longo do tempo: total de issues e taxa de conclusão,
        utilizando eixos Y diferentes para cada métrica.
        
        Returns:
            plotly.graph_objs._figure.Figure: Gráfico de linha com métricas de produtividade da equipe.
        """
        team_metrics = (
            self.merged_data
            .groupby(pd.Grouper(key='created', freq='M'))
            .agg({
                'issue_key': 'count',  # total issues
                'status': lambda x: (x == 'Concluído').sum()  # completed tasks
            })
            .reset_index()
        )
        
        # Calculate completion rate
        team_metrics['completion_rate'] = team_metrics['status'] / team_metrics['issue_key']
        
        fig = make_subplots(specs=[[{"secondary_y": True}]])
        
        fig.add_trace(
            go.Scatter(
                x=team_metrics['created'],
                y=team_metrics['issue_key'],
                name='Total Issues',
                mode='lines+markers'
            ),
            secondary_y=False
        )
        
        fig.add_trace(
            go.Scatter(
                x=team_metrics['created'],
                y=team_metrics['completion_rate'],
                name='Completion Rate',
                mode='lines+markers'
            ),
            secondary_y=True
        )
        
        fig.update_layout(
            title='Team Productivity Timeline',
            xaxis_title='Date',
            hovermode='x unified'
        )
        
        fig.update_yaxes(title_text="Number of Issues", secondary_y=False)
        fig.update_yaxes(
            title_text="Completion Rate", 
            secondary_y=True,
            tickformat='.0%'
        )
        
        return fig

    def plot_workload_analysis(self):
        """Gera análise de sobrecarga de trabalho usando múltiplos indicadores.
        
        Cria uma visualização composta por dois gráficos:
        1. Número de issues atribuídas a cada pessoa ao longo do tempo
        2. Índice de sobrecarga calculado com base em múltiplos indicadores (volume de issues,
           sentimentos negativos e taxa de conclusão)
        
        Returns:
            plotly.graph_objs._figure.Figure: Gráfico composto com análise de sobrecarga de trabalho.
        """
        # Calcular métricas mensais por pessoa
        workload_data = (
            self.merged_data
            .groupby(['assignee', pd.Grouper(key='created', freq='ME')])  # Mudado de 'M' para 'ME'
            .agg({
                'issue_key': 'count',  # total de issues
                'sentiment': lambda x: (x == 'negative').mean(),  # proporção de sentimentos negativos
                'status': lambda x: (x == 'Concluído').mean()  # taxa de conclusão
            })
            .reset_index()
        )
        
        # Criar subplots para diferentes métricas
        fig = make_subplots(
            rows=2, 
            cols=1,
            subplot_titles=(
                'Issues Atribuídas por Pessoa ao Longo do Tempo',
                'Indicadores de Sobrecarga por Pessoa'
            ),
            vertical_spacing=0.22
        )
        
        # 1. Gráfico de linha para issues ao longo do tempo
        for assignee in workload_data['assignee'].unique():
            person_data = workload_data[workload_data['assignee'] == assignee]
            
            fig.add_trace(
                go.Scatter(
                    x=person_data['created'],
                    y=person_data['issue_key'],
                    name=assignee,
                    mode='lines+markers',
                    hovertemplate=(
                        "<b>%{x}</b><br>" +
                        "Issues: %{y}<br>" +
                        "Pessoa: " + assignee
                    )
                ),
                row=1, col=1
            )
        
        # 2. Gráfico de barras para médias gerais
        summary_data = workload_data.groupby('assignee').agg({
            'issue_key': 'mean',  # média de issues por mês
            'sentiment': 'mean',  # média de sentimentos negativos
            'status': 'mean'  # média de taxa de conclusão
        }).reset_index()
        
        # Normalizar os valores para melhor visualização
        summary_data['issue_key_norm'] = summary_data['issue_key'] / summary_data['issue_key'].max()
        summary_data['workload_index'] = (
            (summary_data['issue_key_norm'] * 0.4) + 
            (summary_data['sentiment'] * 0.3) + 
            ((1 - summary_data['status']) * 0.3)  # invertido pois menor taxa de conclusão indica sobrecarga
        )
        
        # Criar uma escala de cores baseada no índice de sobrecarga
        colors = px.colors.diverging.RdYlGn_r
        normalized_indices = (summary_data['workload_index'] - summary_data['workload_index'].min()) / \
                            (summary_data['workload_index'].max() - summary_data['workload_index'].min())
        bar_colors = [colors[int(i * (len(colors)-1))] for i in normalized_indices]
        
        fig.add_trace(
            go.Bar(
                x=summary_data['assignee'],
                y=summary_data['workload_index'],
                marker_color=bar_colors,
                hovertemplate=(
                    "<b>%{x}</b><br>" +
                    "Índice de Sobrecarga: %{y:.2f}<br>" +
                    "Média de Issues/Mês: %{customdata[0]:.1f}<br>" +
                    "Taxa de Sentimentos Negativos: %{customdata[1]:.1%}<br>" +
                    "Taxa de Conclusão: %{customdata[2]:.1%}"
                ),
                customdata=np.column_stack((
                    summary_data['issue_key'],
                    summary_data['sentiment'],
                    summary_data['status']
                ))
            ),
            row=2, col=1
        )
        
        # Atualizar layout
        fig.update_layout(
            height=800,
            title_text='Análise de Sobrecarga de Trabalho da Equipe',
            showlegend=True,
            hovermode='closest'
        )
        
        # Atualizar eixos
        fig.update_xaxes(title_text='Data', row=1, col=1)
        fig.update_xaxes(title_text='Pessoa', row=2, col=1)
        fig.update_yaxes(title_text='Número de Issues', row=1, col=1)
        fig.update_yaxes(title_text='Índice de Sobrecarga', row=2, col=1)
        
        return fig

In [491]:
jira_analysis = JiraSentimentAnalyzer(jira_data, jira_sentiment_data)

In [492]:
jira_analysis.merged_data.head()    

issue_id issue_key project priority     status         assignee  \
0     10074  CSTONE-1  CSTONE   Medium  Concluído     Johny Arriel   
1     10075  CSTONE-2  CSTONE   Medium  Concluído  Luiza Marcondes   
2     10076  CSTONE-3  CSTONE   Medium  Concluído     Johny Arriel   
3     10077  CSTONE-4  CSTONE   Medium  Concluído  Luiza Marcondes   
4     10078  CSTONE-5  CSTONE   Medium  Concluído  Luiza Marcondes   

           creator                          created  \
0  Luiza Marcondes 2024-05-29 14:10:58.526000+00:00   
1  Luiza Marcondes 2024-05-29 14:11:01.651000+00:00   
2  Luiza Marcondes 2024-05-29 14:11:04.461000+00:00   
3  Luiza Marcondes 2024-05-29 14:11:07.557000+00:00   
4  Luiza Marcondes 2024-05-29 14:11:10.063000+00:00   

                      updated  issuetype  \
0  2024-06-10 23:13:30.458+00   História   
1  2024-05-29 14:11:04.596+00  Subtarefa   
2  2024-05-29 14:11:07.681+00  Subtarefa   
3  2024-06-18 18:56:10.673+00  Subtarefa   
4  2024-07-15 12:50:28.082+00  Subtarefa   

                               issuetype_description  \
0  Funcionalidade ou recurso expressado como um o...   
1  Uma pequena parte do trabalho que faz parte de...   
2  Uma pequena parte do trabalho que faz parte de...   
3  Uma pequena parte do trabalho que faz parte de...   
4  Uma pequena parte do trabalho que faz parte de...   

                           summary  \
0              Realizar setup Jira   
1                 Configurar board   
2   Registrar usuários do projeto    
3  Integrar com repositório github   
4           Outras automações Jira   

                                         description  \
0  Como  um membro da equipe responsável pela con...   
1  Configurar e personalizar um novo projeto no J...   
2   Registrar usuários do projeto Stone Lab no Jira.   
3             Fazer integração do Jira com o GitHub.   
4  Alguns itens para explorar: Criação de tarefas...   

                                          all_fields commits  \
0  {"Rank": "0|i000ff:", "Team": null, "votes": {...      []   
1  {"Rank": "0|i000dz:", "Team": null, "votes": {...      []   
2  {"Rank": "0|i000e7:", "Team": null, "votes": {...      []   
3  {"Rank": "0|i000ef:", "Team": null, "votes": {...      []   
4  {"Rank": "0|i000ry:d", "Team": null, "votes": ...      []   

                      time_mined sentiment  confidence  
0  2025-03-24 16:33:47.196763+00  positive        0.90  
1  2025-03-24 16:33:46.385039+00   neutral        0.90  
2  2025-03-24 16:33:45.275883+00   neutral        0.90  
3  2025-03-24 16:33:44.023845+00   neutral        0.90  
4  2025-03-24 16:33:43.237916+00  positive        0.85

In [493]:

fig1 = jira_analysis.plot_overall_sentiment_distribution()
fig1.show()

fig2 = jira_analysis.plot_sentiment_by_issue_type()
fig2.show()

fig3 = jira_analysis.plot_temporal_sentiment_evolution()
fig3.show()

fig4 = jira_analysis.plot_productivity_evolution()
fig4.show()

# Relação carga/conclusão
fig5 = jira_analysis.plot_workload_completion_relation()
fig5.show()

# Evolução dos sentimentos
fig6 = jira_analysis.plot_sentiment_evolution()
fig6.show()

# Média de produtividade
fig7 = jira_analysis.plot_average_productivity()
fig7.show()

fig8 = jira_analysis.plot_team_productivity_timeline()
fig8.show()

fig9 = jira_analysis.plot_workload_analysis()
fig9.show()

C:\Users\breno\AppData\Local\Temp\ipykernel_9000\2263127413.py:117: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.



C:\Users\breno\AppData\Local\Temp\ipykernel_9000\2263127413.py:443: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.

